# Percept perceived data to BIDS conformation

## Import packages

In [8]:
# importing Python and packages

import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np

import scipy
import matplotlib.pyplot as plt
from scipy import signal

import openpyxl
from openpyxl import Workbook, load_workbook
import xlrd

#mne
import mne_bids
from mne_bids import (write_raw_bids, read_raw_bids, write_meg_calibration,
                      write_meg_crosstalk, BIDSPath, print_dir_tree,
                      make_dataset_description)

from mne_bids.stats import count_events
               
import mne
from mne.datasets import sample, eegbci
from mne.time_frequency import tfr_morlet 

from pprint import pprint
import shutil


from importlib import reload          

In [1]:
# imports from Jonathan
# Dependencies to install before use
# For ipyfilechooser and ipywidgets
# conda install -c conda-forge ipywidgets
# conda install -c conda-forge ipyfilechooser
# For TMSiSDK
# download from https://gitlab.com/tmsi/tmsi-python-interface/-/tree/main/TMSiSDK
# and do pip install https://gitlab.com/tmsi/tmsi-python-interface/-/blob/main/requirements_Windows.txt

# Import packages and functions
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser
from BIDS_input_backend import *
# from TMSiSDK.file_readers import Poly5Reader
import ipympl

Using qt as 2D backend.


Get path to the current PyPerceive code folder

In [3]:
# create a path to the PyPerceive_Project folder -> project_path
# and a path to the "Data" folder and "Code" folder within the project
project_path = os.getcwd()
while project_path[-10:] != 'PyPerceive':
    project_path = os.path.dirname(project_path)

code_path = os.path.join(project_path, 'code')
sys.path.append(code_path)

# change directory to code path
os.chdir(code_path)
os.getcwd()

'c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code'

Import internal packages from the folder PerceiveImport

In [5]:
# Import internal packages

from PerceiveImport.classes import main_class, modality_class, metadata_class, session_class, condition_class, task_class
import PerceiveImport.methods.load_matfile as load_matfile
import PerceiveImport.methods.find_folders as find_folders

importlib.reload(main_class)
importlib.reload(session_class)
importlib.reload(task_class)
importlib.reload(condition_class)
importlib.reload(metadata_class)
importlib.reload(modality_class)
importlib.reload(load_matfile)
importlib.reload(find_folders)

<module 'PerceiveImport.methods.find_folders' from 'c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code\\PerceiveImport\\methods\\find_folders.py'>

## Define Subject and File details of the desired Perceive Files
- first load perceived .mat files with mne.io.read_raw_ft()
- then convert mne data to bids
- save bids converted files in rawdata

In [15]:
# define subject and recording details to select and load perceived .mat files

sub024 = main_class.PerceiveData(
    sub = "024", 
    incl_modalities=["survey"],
    incl_session = ["postop"],
    incl_condition =['m0s0'],
    incl_task = ["rest"]
)

Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
LOADED sub024_ses-2021061806255999_run-LMTD20210618062900_1.mat
Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
LOADED sub024_ses-2021061806255999_run-LMTD20210618063200_4.mat
Creating RawArray with float64 data, n_channels=3, n_times=5187
    Range : 0 ... 5186 =      0.000 ...    20.744 secs
Ready.
LOADED sub024_ses-2021061806255999_run-LMTD20210618063300_5.mat
Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
LOADED sub024_ses-2021061806255999_run-LMTD20210618063400_6.mat
Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
LOADED sub024_ses-2021061806255999_run-LMTD20210618063500_7.mat
Creating RawArray with float64 data, n_channels=3,

In [17]:
raw = sub024.survey.postop.m0s0.rest.data['RestBSSuRingL']


Create configuration file with mne_bids_pipeline
- make sure you install mne_bids_pipeline beforehand: conda install -c conda-forge mne-bids-pipeline

In [15]:
code_path

'c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code'

In [ ]:
# use anaconda prompt terminal: 
# mne_bids_pipeline --create-config=c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code\\perceive2BIDS_config.py

### Define bids_root and bids_path
- BIDS root is the root folder of your BIDS dataset
- BIDS path 
- sourcedata is stored in Onedrive: DATA_TEST > perceivedata  

In [26]:
# define bids_root = path to rawdata folder

datapath = os.path.join(find_folders.get_onedrive_path("onedrive")) # path to DATA_TEST

bids_root = os.path.join(datapath, "perceivedata2BIDS", "rawdata")
bids_root

'c:\\Users\\jebe12\\OneDrive - Charité - Universitätsmedizin Berlin\\DATA_TEST\\perceivedata2BIDS\\rawdata'

In [27]:
# define path starting at bids_root, defining BIDS folder structure hierarchy

bids_path = BIDSPath(subject='024', session='EEGLfpMedOff01', task='Rest',
                                 acquisition='StimOff', run='01', datatype='ieeg',
                                 root=bids_root)

bids_path

BIDSPath(
root: c:/Users/jebe12/OneDrive - Charité - Universitätsmedizin Berlin/DATA_TEST/perceivedata2BIDS/rawdata
datatype: ieeg
basename: sub-024_ses-EEGLfpMedOff01_task-Rest_acq-StimOff_run-01)

In [28]:
# create mne-bids folder hierarchies

bids_path = bids_path.mkdir()
print(bids_path.directory)

c:\Users\jebe12\OneDrive - Charité - Universitätsmedizin Berlin\DATA_TEST\perceivedata2BIDS\rawdata\sub-024\ses-EEGLfpMedOff01\ieeg


### Start a new BIDS directory using write_raw_bids()
- this function extracts meta data from raw data 
- and formats it in a BIDS compatible way

In [24]:
print(mne_bids.write_raw_bids.__doc__)

Save raw data to a BIDS-compliant folder structure.

    .. warning:: * The original file is simply copied over if the original
                   file format is BIDS-supported for that datatype. Otherwise,
                   this function will convert to a BIDS-supported file format
                   while warning the user. For EEG and iEEG data, conversion
                   will be to BrainVision format; for MEG, conversion will be
                   to FIFF.

                 * ``mne-bids`` will infer the manufacturer information
                   from the file extension. If your file format is non-standard
                   for the manufacturer, please update the manufacturer field
                   in the sidecars manually.

    Parameters
    ----------
    raw : mne.io.Raw
        The raw data. It must be an instance of `mne.io.Raw` that is not
        already loaded from disk unless ``allow_preload`` is explicitly set
        to ``True``. See warning for the ``allow_preloa

In [33]:
raw.ch_names

['LFP_Stn_L_03',
 'LFP_Stn_L_13',
 'LFP_Stn_L_02',
 'LFP_Stn_L_12',
 'LFP_Stn_L_01',
 'LFP_Stn_L_23']

In [40]:

for ch in raw.ch_names:
    raw.set_channel_types({ch:"eeg"})


In [41]:
# write the rawdata to bids

write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    overwrite=False,
    format="BrainVision",
    allow_preload=True
)

ValueError: The specified datatype ieeg was not found in the raw object. Please specify the correct datatype using `bids_path.update(datatype="<datatype>")` or use raw.set_channel_types to set the correct channel types in the raw object.